In [1]:
import pandas as pd

schools_data_path = 'C:\\Users\\Scott\\Downloads\\UCBBE\\UCBB\\04-pandas\\Homework\\Instructions\\PyCitySchools\\raw_data\\schools_complete.csv'

student_data_path = 'C:\\Users\\Scott\\Downloads\\UCBBE\\UCBB\\04-pandas\\Homework\\Instructions\\PyCitySchools\\raw_data\\students_complete.csv'


In [2]:
data_students = pd.read_csv(student_data_path)
data_schools = pd.read_csv(schools_data_path)

In [3]:
per_passing_math = 100*len(data_students[data_students.math_score >= 70])/len(data_students) 
per_passing_reading = 100*len(data_students[data_students.reading_score >= 70])/len(data_students)
per_passing = (per_passing_math + per_passing_reading)/2

print(data_students.reading_score.min())

#d =  {'id': 'CS2_056', 'cost': 2, 'name': 'Tap'}
#df = pd.DataFrame([d], columns=d.keys())

d =  {'Total Schools': len(data_schools),
      'Total students': len(data_students),
      'Total budget': data_schools.budget.sum(),
      'Average math score': data_students.math_score.mean(),
      'Percent math passing': per_passing_math,
      'Percent reading passing': per_passing_reading,
      'Percent passing':per_passing}

summary_df = pd.DataFrame([d], columns=d.keys())
                                            
summary_df.style.format({'Total budget':'${:,.0f}'.format,
                         'Percent math passing':'%{:,.1f}'.format,
                         'Percent reading passing':'%{:,.1f}'.format,
                         'Percent passing':'%{:,.1f}'.format,})
                                            

63


In [4]:
#print(data_students.keys())
#print(data_schools.keys())
print(data_students.head())
#print(data_schools.head)

   Student ID               name gender grade             school  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  
0             66          79  
1             94          61  
2             90          60  
3             67          58  
4             97          84  


In [5]:
#Summary per school section
data_schools['Per Student Budget']=data_schools['budget']/data_schools['size']

#create a column for math pass = TRUE
data_students['math_pass']=data_students['math_score'] >= 70 
data_students['reading_pass']=data_students['reading_score'] >= 70 

#Find pass/fail mean
school_math_per_pass =  data_students.groupby(['school'])['math_pass'].mean()*100
school_reading_per_pass = data_students.groupby(['school'])['reading_pass'].mean()*100
school_overall_per_pass = (school_math_per_pass+school_reading_per_pass)/2

#find the means by the school
school_math_average =  data_students.groupby(['school'])['math_score'].mean()
school_reading_average = data_students.groupby(['school'])['reading_score'].mean()

#convert pandas seriers to dataFrame
school_math_average_df =  school_math_average.to_frame(name='math_score_average')
school_reading_average_df =  school_reading_average.to_frame(name='reading_score_average')
school_math_per_pass_df = school_math_per_pass.to_frame(name='math_per_pass')
school_reading_per_pass_df = school_reading_per_pass.to_frame(name='reading_per_pass')
school_overall_per_pass_df = school_overall_per_pass.to_frame(name='overall_per_pass')

#copy index as school to it's own column for tracking
school_math_average_df['school_m'] = school_math_average_df.index 
school_reading_average_df['school_r'] = school_reading_average_df.index
school_math_per_pass_df['school_m_p_pass'] = school_math_per_pass_df.index
school_reading_per_pass_df['school_r_p_pass'] = school_reading_per_pass_df.index
school_overall_per_pass_df['school_overall_p_pass'] = school_overall_per_pass_df.index

#the great merge
school_data_1 = pd.merge(data_schools,school_math_average_df,left_on='name',right_on='school_m')
school_data_1 = pd.merge(school_data_1,school_reading_average_df,left_on='name',right_on='school_r')
school_data_1 = pd.merge(school_data_1,school_math_per_pass_df,left_on='name',right_on='school_m_p_pass')
school_data_1 = pd.merge(school_data_1,school_reading_per_pass_df,left_on='name',right_on='school_r_p_pass')
school_data_1 = pd.merge(school_data_1,school_overall_per_pass_df,left_on='name',right_on='school_overall_p_pass')

#clean up the extra school name columns
#print(school_data_1.columns.tolist())
del_col = ['school_m','school_r','school_m_p_pass','school_overall_p_pass','school_r_p_pass']
school_data_1.drop(del_col,axis=1,inplace=True)

school_data_1.style.format({'budget':'${:,.0f}'.format,
                         'math_per_pass':'%{:,.1f}'.format,
                         'reading_per_pass':'%{:,.1f}'.format,
                         'overall_per_pass':'%{:,.1f}'.format,})


In [6]:
print('Top 5 performing schools')
school_data_1.sort_values(['overall_per_pass'],ascending=False).head(5)

Top 5 performing schools


,School ID,name,type,size,budget,Per Student Budget,math_score_average,reading_score_average,math_per_pass,reading_per_pass,overall_per_pass
6,6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
14,14,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
9,9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
4,4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [7]:
print('Bottom 5 performing schools')
school_data_1.sort_values(['overall_per_pass'],ascending=True).head(5)

Bottom 5 performing schools


,School ID,name,type,size,budget,Per Student Budget,math_score_average,reading_score_average,math_per_pass,reading_per_pass,overall_per_pass
11,11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
1,1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
0,0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
12,12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
13,13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [8]:
#group by school and grade then take the mean of math_scote
math_by_grade = data_students.groupby(['school','grade'])['math_score'].mean()

#mess with dataframe layout to get something presentable
math_by_grade_unstacked = math_by_grade.unstack(level=1)
cols = [ '9th','10th', '11th', '12th']
math_by_grade_unstacked = math_by_grade_unstacked[cols]

#final output
print('Math scores per grade level and school')
math_by_grade_unstacked.style.format({'9th':'{:,.1f}'.format,
                                     '10th':'{:,.1f}'.format,
                                     '11th':'{:,.1f}'.format,
                                     '12th':'{:,.1f}'.format})


Math scores per grade level and school


In [9]:
#group by school and grade then take the mean of reading_scote
math_by_grade = data_students.groupby(['school','grade'])['reading_score'].mean()

#mess with dataframe layout to get something presentable
math_by_grade_unstacked = math_by_grade.unstack(level=1)
cols = [ '9th','10th', '11th', '12th']
math_by_grade_unstacked = math_by_grade_unstacked[cols]

#final output
print('Reading scores per grade level and school')
math_by_grade_unstacked.style.format({'9th':'{:,.1f}'.format,
                                     '10th':'{:,.1f}'.format,
                                     '11th':'{:,.1f}'.format,
                                     '12th':'{:,.1f}'.format})

Reading scores per grade level and school


In [10]:
#%matplotlib inline
#school_data_1['Per Student Budget'].hist()

In [11]:
import numpy as np
bins = np.linspace(school_data_1['Per Student Budget'].min(),school_data_1['Per Student Budget'].max(),num=5)
group_labels = ['$'+str(round(bins[0])) + ' to ' + str(round(bins[1])),
                '$'+str(round(bins[1])) + ' to ' + str(round(bins[2])),
                '$'+str(round(bins[2])) + ' to ' + str(round(bins[3])),
                '$'+str(round(bins[3])) + ' to ' + str(round(bins[4]))]


school_data_1['Spending Ranges (Per Student)'] = pd.cut( school_data_1['Per Student Budget'],bins, labels=group_labels)

# Create a GroupBy object based upon "View Group"
school_budget_group = school_data_1.groupby('Spending Ranges (Per Student)')

#average the catagories
temp_df = school_budget_group['math_score_average','reading_score_average',
                          'math_per_pass','reading_per_pass','overall_per_pass'].mean()
#
print('Spending per student and outcome')
temp_df.style.format({            'math_score_average':'{:,.1f}'.format,
                                     'reading_score_average':'{:,.1f}'.format,
                                             'math_per_pass':'{:,.1f}'.format,
                                          'reading_per_pass':'{:,.1f}'.format,
                                          'overall_per_pass':'{:,.1f}'.format})


Spending per student and outcome


In [12]:
#school_data_1['size'].hist()

In [13]:
import numpy as np
bins = np.linspace(school_data_1['size'].min(),school_data_1['size'].max(),num=4)
group_labels = ['Small ('+str(round(bins[0])) + ' to ' + str(round(bins[1])) + ')',
                'Medium ('+str(round(bins[1])) + ' to ' + str(round(bins[2]))+ ')',
                'Large ('+str(round(bins[2])) + ' to ' + str(round(bins[3]))+ ')']


school_data_1['Size range'] = pd.cut( school_data_1['size'],bins, labels=group_labels)

# Create a GroupBy object based upon "View Group"
school_size_group = school_data_1.groupby('Size range')

#average the catagories
temp_df = school_size_group['math_score_average','reading_score_average',
                          'math_per_pass','reading_per_pass','overall_per_pass'].mean()
temp_df['size_group_count']=school_size_group['Size range'].count()
#
print('School size and outcome')
temp_df.style.format({            'math_score_average':'{:,.1f}'.format,
                                     'reading_score_average':'{:,.1f}'.format,
                                             'math_per_pass':'{:,.1f}'.format,
                                          'reading_per_pass':'{:,.1f}'.format,
                                          'overall_per_pass':'{:,.1f}'.format})

School size and outcome


In [14]:
school_type_group = school_data_1.groupby('type')

#average the catagories
temp_df = school_type_group['math_score_average','reading_score_average',
                          'math_per_pass','reading_per_pass','overall_per_pass'].mean()

print('School size and outcome')
temp_df.style.format({            'math_score_average':'{:,.1f}'.format,
                                     'reading_score_average':'{:,.1f}'.format,
                                             'math_per_pass':'{:,.1f}'.format,
                                          'reading_per_pass':'{:,.1f}'.format,
                                          'overall_per_pass':'{:,.1f}'.format})

School size and outcome


In [15]:
# Observations
# 1) Charter schools outperform District schools by a wide margin. As with most data sets, sometimes the data doesn't fully 
#   capture the environment.  In this case what is not captured in data is the amount of parental involvement.  From other 
#   sets, and personal observation, Charter schools are not the default school, so parents have to actively work to get
#   their child into a Charted school.  These are the same parents that are generally more involved at home also. The 
#   Charter schools have parents that have self selected. 
# 2) Just having more money does get better results. But this could be a false conclusion because we don't know the
#    of student at each school.  For example some schools cater to single mothers and provide day care for the mothers.
#    This is a different situation than a school with kids from 40 year old parents with PhD's.
# 3) Schools perform pretty much the same on reading, and distinguish themselves on math.
